In [ ]:
!pip install pandas
!pip install windrose
!pip install matplotlib
!pip install selenium=3.141

# IMPORTANDO BIBLIOTECAS E FUNÇÕES AUXILIARES

In [1]:
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from windrose import WindroseAxes           # https://windrose.readthedocs.io/en/latest/usage.html
import matplotlib.ticker as tkr
from Rdita_Atrs_2024 import *               # DESENVOLVIDO POR JOHN HEBERTY DE FREITAS - FUNÇÕES AUXILIARES
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import json
import math

from Modulos.BROWSER.Engine import *
from Modulos.DADOS.Engine import *
from Functions import *
from Script1 import *
from Default import *

%matplotlib inline

# ABRINDO O DADO DOS VENTOS
DataFilesAuto = ReadDatasets(WeatherStationsPath, SaveAnalysis=WeatherStationsPath_OK)

# ABRINDO NAVEGADOR PARA OBTER A DECLINAÇÃO MAGNETICA
driver = OpenBrowser(url=Url_MagneticDeclination)

# BUSCANDO RUMO QUE MÁXIMIZA O FO

In [2]:

# PERCORRENDO OS DADOS DE CADA AEROPORTOE OTIMIZANDO VALORES
TabelaFinal = []
for aeroporto, ideaeroporto in zip(DataFilesAuto, range(1,len(list(DataFilesAuto.keys()))+1)):

    print(f"AEROPORTO: {aeroporto} - #/$".replace("#",str(ideaeroporto)).replace("$",str(len(list(DataFilesAuto)))))
    
    # OBTENDO OS DADOS DO AEROPORTO
    Info = DataFilesAuto[aeroporto]
    tabelao = Info["Data"].copy()
    local = Info["Local"]
    
    # FILTRANDO DADOS PARA ANALISES P/5 ANOS E 10 ANOS
    tabelao5    = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*5 )].reset_index(drop=True).copy()
    tabelao10   = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*10)].reset_index(drop=True).copy()
    tabelao15   = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*15)].reset_index(drop=True).copy()
    tabelao20   = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*20)].reset_index(drop=True).copy()
    tabelao25   = tabelao[tabelao["DATA"]>=tabelao["DATA"].max()-timedelta(days=365*25)].reset_index(drop=True).copy()
    
    # OBTENDO A DECLINAÇÃO MAGNETICA
    MagneticDeclination = round(GetMagneticDeclination(local[0], local[1], driver), DecimalPlaces)
    
    # IMAGEM EM BRANCO
    imagem_branca = np.zeros((Height_IMG, Width_IMG, 3), dtype=np.uint8)

    # Calculando as coordenadas do centro da imagem
    centro_x = int(Width_IMG / 2)
    centro_y = int(Height_IMG / 2)

    maior = max(LIMITES)
    menor = min(LIMITES)

    proporcao = 0.24/maior
    comprimento = int(Width_IMG * (maior * proporcao))
    WidthRunway = 0
    for limite in LIMITES[::-1]:
        r = int(Width_IMG * (limite * proporcao))
        if limite == LIMITES[-1]: cv.circle(imagem_branca, (centro_x, centro_y), r, (255, 255, 255), -1)
        cv.circle(imagem_branca, (centro_x, centro_y), r, (0, 0, 255), 2)
        if limite == WindRunwayLimite: WidthRunway = r

    # RETAS
    if True:
        
        # Desenhando as linhas radiais para cada direção
        graus_rumo = calcular_setores(RosadosVentos, SectorNames[RosadosVentos])
        for name, angulo in graus_rumo.items(): DrawRadialLine(imagem_branca, (centro_x, centro_y), comprimento, angulo[0], (0, 0, 0), 2)

    # IDENTIFICANDO CADA AREA DA ROSA DOS VENTOS COM UMA COR ESPECIFICA
    if True:
        imagem_AREA = imagem_branca.copy()

        # Crie uma máscara para identificar as regiões brancas na imagem
        branco_baixo = np.array([255, 255, 255])
        branco_alto = np.array([255, 255, 255])
        mascara_branco = cv.inRange(imagem_AREA, branco_baixo, branco_alto)

        # Encontre os contornos na imagem mascarada
        contornos, _ = cv.findContours(mascara_branco, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

        # REALIZANDO O AGRUPAMENTO DAS AREAS
        rotulos_clusters, grupos, center_grups = Agroup(contornos)

        # Criar uma nova imagem para desenhar os contornos coloridos
        cores_grupos = GenerateUniqueGrayColors(len(LIMITES)*RosadosVentos)

        # SALVANDO INFORMAÇÕES SOBRE AS AREAS
        dict_area_info = {}

        # Para cada grupo de contornos, criar uma paleta de cores única
        contagem = 0
        
        # ORDENANDO DOS MAIORES GRUPOS PARA OS MENORES 
        grupos = sorted(grupos, key=lambda x: cv.contourArea(x[0]), reverse=True)
        id_grupo = 1
        for grupo, limite in zip(grupos, LIMITES[::-1]):
            # Gerar cores únicas para o grupo com base no número de contornos
            parcial = {}
            for contorno in grupo:
                cor = cores_grupos[contagem]  # Selecionar uma cor única da paleta para cada área dentro do grupo
                cv.drawContours(imagem_AREA, [contorno], -1, cor, thickness=cv.FILLED)  # Preencher o contorno com a cor
        
                # # CALCULANDO ANGULO DO CENTRO PARA AREA PARA IDENTIFICAR O RUMO
                p1 = BaricentroArea(contorno)
                p2 = centro_x, centro_y
                angulo = CalculateAzimuth(p1, p2)
                rumo = ""
                for name, angulos in graus_rumo.items():
                    maior_ = max(angulos)
                    menor_ = min(angulos)
                    if angulo >= menor_ and angulo <= maior_:
                        rumo = name
                    elif angulo <= menor_ and angulo <= maior_:
                        rumo = "N"

                contagem+=1
                
                parcial[rumo] = {
                    # "CONTORNO": contorno, 
                    "AREA": cv.contourArea(contorno), 
                    "COR": cor,
                    "LIMITE": limite
                }
            dict_area_info[id_grupo] = parcial
            id_grupo+=1
    
    # DESENHANDO O RETANGULO DA PISTA E RECORTANDO
    if True:

        # Definindo as dimensões do retângulo
        largura_retangulo = WidthRunway * 2
        altura_retangulo = comprimento * 2

        # Calculando os vértices do retângulo
        vertices = np.array([
            [centro_x - largura_retangulo / 2, centro_y - altura_retangulo / 2],
            [centro_x + largura_retangulo / 2, centro_y - altura_retangulo / 2],
            [centro_x + largura_retangulo / 2, centro_y + altura_retangulo / 2],
            [centro_x - largura_retangulo / 2, centro_y + altura_retangulo / 2]
        ], dtype=np.float32)

        # CRIANDO PONTO DE RESTAURAÇÃO DA IMG SEM DESENHO
        imagem_AREA_BK = imagem_AREA.copy()
        
        # PERCORRENDO DADOS
        DFS = {
            ">=5"   :tabelao5,
            ">=10"  :tabelao10,
            ">=15"  :tabelao15,
            ">=20"  :tabelao20
        }
        dict_final = {}
        dict_pacial = {}
        for Name in DFS:
            Dataset = DFS[Name]
            
            # LIMPANDO A PASTA DE IMGS SE FOR FAZER UM NOVO VIDEO
            if MakeVideo is True: ClearFolder(FolderImages)
            
            # SE EXISTIR DADOS PROSSIGA
            if len(Dataset) > 0:
            
                # CONTABILIZANDO VENTOS PARA CADA DIRECAO EM PORCENTAGEM
                df_pct_ventos = Script1(Dataset[DirectionName], Dataset[WindName], SectorNames[RosadosVentos], LIMITES)
                
                # DADOS PARA SALVAR A OTIMIZAÇÃO
                FU_FINAL = 0
                FU_FINAL_MAXIMO = 0
                FU_FINAL_GRAU = 0
                
                # ANGULANDO A PISTA PARA BUSCAR OTIMIZAR O FU
                for grau in range(0, 181):
                    
                    # RESTAURANDO BK DA IMG SEM DESENHOS
                    imagem_AREA = imagem_AREA_BK.copy()
                    
                    # Criando uma matriz de rotação de 30 graus em torno do centro da imagem
                    theta = np.radians(grau)
                    rot_mat = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
                    centro_retangulo = np.array([centro_x, centro_y])
                    vertices_rotacionados = np.dot(vertices - centro_retangulo, rot_mat.T) + centro_retangulo

                    cor_pista = (255, 255, 255)
                    # Desenhando o retângulo na imagem
                    cv.polylines(imagem_AREA, [vertices_rotacionados.astype(np.int32)], isClosed=True, color=cor_pista, thickness=2)
                    
                    # Crie uma máscara para identificar as regiões brancas na imagem
                    mascara = cv.inRange(imagem_AREA, cor_pista, cor_pista)

                    # Encontre os contornos na imagem mascarada
                    contornos, _ = cv.findContours(mascara, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
                    
                    # PINTANDO A PISTA COMPLETA DE UMA COR PARA IDENTIFICAR POR MASCARA E RECORTAR
                    imagem_AREA2 = imagem_AREA_BK.copy()
                    cv.drawContours(imagem_AREA2, contornos, -1, cor_pista, thickness=cv.FILLED)
                    
                    # Aplicar a máscara na imagem original
                    mascara_recorte = cv.inRange(imagem_AREA2, cor_pista, cor_pista)
                    pista_recortada = cv.bitwise_and(imagem_AREA, imagem_AREA, mask=mascara_recorte)
                    
                    # PUXANDO TABELA DE VENTOS DO SCRIPT 1 E RENOMEANDO AS ValoresS 
                    # PARA UMA CONSULTA MAIS FACILITADA
                    df_pct_ventos_cp = df_pct_ventos.copy()
                    df_pct_ventos_cp.columns = LIMITES + ["40+"]

                    # FIXANDO A PISTA DE MAIOR THETA
                    if FU_FINAL > FU_FINAL_MAXIMO:
                        FU_FINAL_MAXIMO = FU_FINAL
                        FU_FINAL_GRAU = grau
                    
                    # SE EXISTIR UM FU DESENHE 
                    if FU_FINAL_MAXIMO > 0:
                                
                        # Criando uma matriz de rotação de 30 graus em torno do centro da imagem
                        theta = np.radians(FU_FINAL_GRAU)
                        rot_mat = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
                        centro_retangulo = np.array([centro_x, centro_y])
                        vertices_rotacionados = np.dot(vertices - centro_retangulo, rot_mat.T) + centro_retangulo
                        
                        cor_pista = (0, 255, 0)
                        # Desenhando o retângulo na imagem
                        cv.polylines(imagem_AREA, [vertices_rotacionados.astype(np.int32)], isClosed=True, color=cor_pista, thickness=2)

                        # DESENHANDO REFERENCIA DOS DADOS
                        DrawReferenceRUNWAY(imagem_AREA, (centro_x, centro_y), comprimento, FU_FINAL_GRAU, (255, 165, 0), 25)
                        
                    # ANALISANDO O FATOR DE ULILIZAÇÃO DA PISTA ATUAL
                    FU_FINAL = 0
                    for id_grupo in dict_area_info:
                        grupo = dict_area_info[id_grupo]
                        for rumo in grupo:
                            Info = grupo[rumo]
                            cor_grupo = Info["COR"]
                            limite = Info["LIMITE"]
                            
                            # Crie uma máscara para identificar as regiões brancas na imagem
                            mascara = cv.inRange(pista_recortada.copy(), cor_grupo, cor_grupo)
                            
                            # Encontre os contornos na imagem mascarada
                            contorno, _ = cv.findContours(mascara, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
                            contorno = list(contorno)
                            
                            # NEM TODA AERA ESTA DENTRO DA PISTA LOGO 
                            if len(contorno) > 0:
                                contorno = contorno[0]
                                # IDENTIFICANDO AREA DO CONTORNO DENTRO DA PISTA
                                area_dentro_PPD = cv.contourArea(contorno)
                            
                            # CASO NÃO ESTIVER SETE ZERO
                            else:
                                area_dentro_PPD = 0
                            
                            # FAZENDO A FRAÇÃO DOS VENTROS DENTRO DA PISTA
                            fracao_dentro = (area_dentro_PPD / Info["AREA"]) * df_pct_ventos_cp.loc[rumo][limite]
                            
                            FU_FINAL = FU_FINAL + fracao_dentro
                    FU_FINAL = round(FU_FINAL, 4)

                    # INFORMAÇÕES SOBRE A PISTA ATUAL
                    if True:
                        cor_pista_atual = (255, 255, 255)
                        
                        # INFORMAÇÕES SOBRE A PISTA ATUAL
                        texto = f'DIRECTION NOW' 
                        cv.putText(imagem_AREA, texto, (StartLegendRight, HeightEspaceLegend*1), Fonte, FonteSize, cor_pista_atual, FonteThickness, cv.LINE_AA)
                        
                        texto = f'FO: {FU_FINAL}%'
                        cv.putText(imagem_AREA, texto, (StartLegendRight, HeightEspaceLegend*2), Fonte, FonteSize, cor_pista_atual, FonteThickness, cv.LINE_AA)
                        
                        # pista = HeadboardRunway(grau)
                        texto = f'RUMO: {grau}'
                        cv.putText(imagem_AREA, texto, (StartLegendRight, HeightEspaceLegend*3), Fonte, FonteSize, cor_pista_atual, FonteThickness, cv.LINE_AA)
                                                            
                        texto = f'MAGNETIC DECLINATION: {MagneticDeclination}'
                        cv.putText(imagem_AREA, texto, (StartLegendRight, HeightEspaceLegend*4), Fonte, FonteSize, cor_pista_atual, FonteThickness, cv.LINE_AA)
                            
                        RUNWAY_ORIENTATION = HeadboardRunway(round(grau + MagneticDeclination, DecimalPlaces))
                        texto = f'RUNWAY ORIENTATION: {RUNWAY_ORIENTATION}'
                        cv.putText(imagem_AREA, texto, (StartLegendRight, HeightEspaceLegend*5), Fonte, FonteSize, cor_pista_atual, FonteThickness, cv.LINE_AA)
                    
                        # INFORMAÇÕES SOBRE OS VENTOS DE TRAVEZ ESQUERDO
                        texto = f'CROSS WIND: {round(100 - FU_FINAL, DecimalPlaces)}%'
                        cv.putText(imagem_AREA, texto, (StartLegendRight, HeightEspaceLegend*6), Fonte, FonteSize, cor_pista_atual, FonteThickness, cv.LINE_AA)
                    
                    # INFORMAÇÕES SOBRE A MELHOR PISTA
                    if FU_FINAL_MAXIMO > 0:
                        cor_pista = (0, 255, 0)
                        StartLegendLeft = 40
                        
                        # INFORMAÇÕES SOBRE A PISTA ATUAL
                        texto = f'BEST DIRECTION'
                        cv.putText(imagem_AREA, texto, (StartLegendLeft, HeightEspaceLegend*1), Fonte, FonteSize, cor_pista, FonteThickness, cv.LINE_AA)
                        
                        texto = f'FO: {FU_FINAL_MAXIMO}%'
                        cv.putText(imagem_AREA, texto, (StartLegendLeft, HeightEspaceLegend*2), Fonte, FonteSize, cor_pista, FonteThickness, cv.LINE_AA)
                        
                        texto = f'RUMO: {FU_FINAL_GRAU}'
                        cv.putText(imagem_AREA, texto, (StartLegendLeft, HeightEspaceLegend*3), Fonte, FonteSize, cor_pista, FonteThickness, cv.LINE_AA)
                                                            
                        texto = f'MAGNETIC DECLINATION: {MagneticDeclination}'
                        cv.putText(imagem_AREA, texto, (StartLegendLeft, HeightEspaceLegend*4), Fonte, FonteSize, cor_pista, FonteThickness, cv.LINE_AA)
                       
                        RUNWAY_ORIENTATION = HeadboardRunway( round(FU_FINAL_GRAU + MagneticDeclination, DecimalPlaces))
                        texto = f'RUNWAY ORIENTATION: {RUNWAY_ORIENTATION}'
                        cv.putText(imagem_AREA, texto, (StartLegendLeft, HeightEspaceLegend*5), Fonte, FonteSize, cor_pista, FonteThickness, cv.LINE_AA)
                        
                        # INFORMAÇÕES SOBRE OS VENTOS DE TRAVEZ ESQUERDO
                        texto = f'CROSS WIND: {round(100 - FU_FINAL_MAXIMO, DecimalPlaces)}%'
                        cv.putText(imagem_AREA, texto, (StartLegendLeft, HeightEspaceLegend*6), Fonte, FonteSize, cor_pista, FonteThickness, cv.LINE_AA)
                    
                        # INFORMAÇÕES SOBRE OS VENTOS DE TRAVEZ ESQUERDO
                        texto = f'Airport: {aeroporto}'
                        cv.putText(imagem_AREA, texto, (StartLegendLeft, Height_IMG-HeightEspaceLegend*2), Fonte, FonteSize, cor_pista, FonteThickness, cv.LINE_AA)

                        # INFORMAÇÕES SOBRE OS VENTOS DE TRAVEZ ESQUERDO
                        texto = f'Lat: {local[0]} / Lon: {local[1]}'
                        cv.putText(imagem_AREA, texto, (StartLegendLeft, Height_IMG-HeightEspaceLegend), Fonte, FonteSize, cor_pista, FonteThickness, cv.LINE_AA)

                    # DESENHANDO REFERENCIA DOS DADOS
                    DrawReferenceRUNWAY(imagem_AREA, (centro_x, centro_y), comprimento, grau, (255, 165, 0), 25)

                    if MakeVideo is True:
                        
                        # SALVANDO IMAGEM PARA O VIDEO FINAL
                        cv.imwrite(os.path.join(FolderImages, f"IMG{grau}.jpg"), imagem_AREA)

                dict_pacial[Name] = {
                    "FO": FU_FINAL_MAXIMO,
                    "RUMO": FU_FINAL_GRAU,
                    "CROSS WIND": 1 - FU_FINAL_MAXIMO,
                }
                
                # SE ATIVADO CRIA O VIDEO
                numbers = "".join([row for row in Name if row.isdigit()])
                if MakeVideo is True: 
                    path_save = caminho_saida_video.format(aeroporto, numbers)
                    if os.path.exists(path_save) is False:
                        os.makedirs(path_save)
                    CreateVideo(FolderImages, path_save.format(numbers))

        dict_final[aeroporto] = dict_pacial
        TabelaFinal.append(dict_final)
        with open("TabelaFinal", "w", encoding="UTF-8") as file:
            json.dump(TabelaFinal, file)
    

AEROPORTO: BRASILIA - 1/10


In [ ]:
path_save = caminho_saida_video.format(aeroporto, numbers)
path_save

In [ ]:
path_save

In [ ]:
caminho_saida_video

In [ ]:
FolderImages

In [ ]:
caminho_saida_video.format(aeroporto+Name)

# CRIANDO A WINDROSE

In [ ]:
# Criar uma figura e adicionar os eixos para a rosa-dos-ventos
figura = plt.figure(figsize=(12, 8))
rict_rectangle = [0, 0, 1, 1]
ax = WindroseAxes(figura, rict_rectangle)
figura.add_axes(ax)

# Definir os dados de direção e magnitude do vento
direcao = tabelao[[column for column in tabelao.columns if DirectionName in column][0]]  # Graus de direção do vento
magnetude = tabelao[[column for column in tabelao.columns if WindName in column][0]]  # Magnitude do vento

# Definir algumas configurações opcionais para o gráfico
displayed = True
divisores_dados = [0,3,13,20,25,40] # bins (array 1D ou inteiro, opcional) – número de bins ou uma variável de sequência de bins. Se não definido, bins=6 entre min(var) e max(var).
blowto = False # blowto (bool, opcional.) – se True, a rosa-dos-ventos será pi girada, para mostrar para onde o vento sopra (útil para rosa poluente).
divisores_angular = 8 # Se não definido, nsetores=16, cada setor será 360/16=22,5° e a tabela computada resultante será alinhada com os pontos cardeais.
palette_color = [
    'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r',
    'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r',
    'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r',
    'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r',
    'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r',
    'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r',
    'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r',
    'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r',
    'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r',
    'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn',
    'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r',
    'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone',
    'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool',
    'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'crest', 'crest_r',
    'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'flare', 'flare_r', 'gist_earth',
    'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar',
    'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg',
    'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot',
    'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r', 'inferno', 'inferno_r', 'jet', 'jet_r',
    'magma', 'magma_r', 'mako', 'mako_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r',
    'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket',
    'rocket_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10',
    'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain',
    'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted',
    'twilight_shifted_r', 'viridis', 'viridis_r', 'vlag', 'vlag_r', 'winter', 'winter_r'
]
map_color = plt.get_cmap("Accent")
opening = 0.8 # ( float, opcional ) – entre 0,0 e 1,0, para controlar o espaço entre cada setor (1,0 para nenhum espaço)

# Criar o gráfico da rosa-dos-ventos
#ax.contour(direcao, magnetude, normed=displayed, bins=divisores_dados, nsector=divisores_angular, cmap=map_color)#, opening=opening)
ax.bar(direcao, magnetude, normed=displayed, bins=divisores_dados, blowto=blowto, nsector=divisores_angular, cmap=map_color, opening=opening)

# Configurar os eixos e legendas
ax.tick_params(axis="both", which="major", labelsize=20)  # Configurar os tamanhos das fontes nos eixos
ax.legend(loc="center left", decimal_places=1, bbox_to_anchor=(1.12, 0.5), fontsize=20)  # Adicionar uma legenda ao gráfico
ax.set_rlabel_position(125)  # Posição dos rótulos dos eixos radiais
ax.yaxis.set_major_formatter(tkr.FormatStrFormatter("%2.2f"))  # Formato dos rótulos dos eixos radiais
